In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


Load files

In [2]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')

test_data = pd.read_csv('/kaggle/input/titanic/test.csv')

train_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


Look for missing values

In [3]:
train_data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
test_data.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [5]:
train_data.columns.tolist()

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [6]:
train_data.Age.mean()

29.69911764705882

In [7]:
from sklearn.model_selection import train_test_split


train_data['Age']=train_data['Age'].fillna(train_data['Age'].median()) #use mean for missing values
train_data.loc[train_data[train_data.Embarked.isna()].index.tolist(),'Embarked']='S'

test_data.Age = test_data.Age.fillna(test_data.Age.median())
test_data.Fare=test_data.Fare.fillna(test_data.Fare.median())



features=['Pclass','Sex','Age','SibSp','Parch','Fare']

X=pd.get_dummies(train_data[features])
y=train_data.Survived

test_X=pd.get_dummies(test_data[features])

train_X,val_X,train_y,val_y=train_test_split(X,y,random_state=0)

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

forest_model = RandomForestClassifier(random_state=0)
forest_model.fit(train_X,train_y)

predict_val = forest_model.predict(val_X)
print(accuracy_score(val_y,predict_val)*100,'%')

82.95964125560538 %


In [9]:
from sklearn.neural_network import MLPClassifier

mlp_model = MLPClassifier(random_state=0,learning_rate_init=0.001,max_iter=500)
mlp_model.fit(train_X,train_y)

predict_val = mlp_model.predict(val_X)
print(accuracy_score(val_y,predict_val)*100,'%')

81.16591928251121 %


In [10]:
full_forest_model = RandomForestClassifier(random_state=0)
full_forest_model.fit(X,y)

predict_test = full_forest_model.predict(test_X)

In [11]:
submission = pd.DataFrame({'PassengerId':test_data.PassengerId,'Survived':predict_test})
submission.to_csv('submission.csv',index=False)